<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
%%capture

!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [3]:
%%capture
# Install the vncorenlp python wrapper
!pip install vncorenlp
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [4]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

In [5]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [6]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}

In [7]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/088978.txt.seg',
 'original': 'Xe 30 chỗ sau khi húc đuôi xe_khách bị nạn . Sáng 5/12 , xe_khách 30 chỗ chạy trên Xa_lộ Hà_Nội , hướng từ quận 2 về trung_tâm TP HCM . Đến giữa cầu Sài_Gòn ( quận Bình_Thạnh ) ôtô bị nổ lốp trước , dừng sát dải_phân_cách chờ cứu_hộ . Một lúc sau ôtô chở hơn hai chục khách từ phía sau lao đến , húc mạnh vào đuôi xe bị nạn . Cú tông khiến hành_khách hoảng_loạn la_hét nhưng không ai bị_thương , 2 xe hư_hỏng nặng , kính vỡ vung_vãi hiện_trường ... Tai_nạn xảy ra vào giờ cao_điểm sáng khiến giao_thông qua cầu Sài_Gòn kẹt cứng , kéo_dài hơn một km . Nhiều xe_máy chạy vào làn ôtô nhưng một phần làn_đường bị phong_toả để CSGT xử_lý hiện_trường . Cầu huyết_mạch dẫn vào thành_phố kẹt cứng sau tai_nạn . Sơn_Hoà   Xe 30 chỗ sau khi húc đuôi xe_khách bị nạn . ',
 'summary': 'Ôtô bị nổ lốp trên cầu Sài_Gòn , quận Bình_Thạnh , đang chờ cứu_hộ thì bị xe_khách đâm vào đuôi khiến giao_thông qua khu_vực kẹt cứng hơn m

In [8]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [8]:
train_df = get_dataframe(train_paths)

In [9]:
test_df = get_dataframe(test_paths)
val_df = get_dataframe(val_paths)

In [ ]:
train_df.head(2)

,file,original,summary
0,/content/vietnews-master/data/train_tokenized/...,"Phản_ánh đến Tuổi_Trẻ Online , chị Nguyễn_Thị_...",Lô hàng trị_giá hơn 100 triệu đồng từ Singapor...
1,/content/vietnews-master/data/train_tokenized/...,"Ngày 29/5 , rapper PSY lên_tiếng giải_thích về...","Chủ_nhân ca_khúc "" Gangnam_Style "" thừa_nhận t..."


## word to vec

In [ ]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
len(sentence_vectors)

16

## word to vec (vietnamese)

In [10]:
!pip install -q pyvi

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Read embedding
word_dict = []
embeddings_index = {}
embedding_dim = 300
max_feature = len(embeddings_index) + 2

f = open('/content/drive/MyDrive/nam3/course/nlp/W2V_C2V_ner.vec')
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        pass
f.close()

print('Embedding data loaded')

Embedding data loaded


The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
import networkx as nx

# nx_graph = nx.from_numpy_array(sim_mat)
# scores = nx.pagerank(nx_graph, max_iter=100)

In [15]:
!pip install -q rouge-score

In [19]:
sentences = []

for s in test_df.original.head(1):
  sentences.append(sent_tokenize(s))  

sentences

[['Đại_tá Lê_Ngọc , Trưởng_phòng CSGT TP Đà_Nẵng , đơn_vị đã chính_thức đưa vào hoạt_động trang Facebook có địa_chỉ : Cảnh_sát giao_thông CATP Đà_Nẵng .',
  'Trước đó từ ngày 1/3 , CSGT Đà_Nẵng đã lập trang Facebook này nhưng làm thí_điểm để tuyên_truyền , hướng_dẫn luật giao_thông cho những người thường_xuyên la_cà quán_xá uống cà_phê , chơi Facebook .',
  'Lập Facebook , CSGT hy_vọng sẽ tiếp_nhận thông_tin và tương_tác nhiều hơn với người_dân .',
  'Sau một thời_gian thấy hiệu_quả , Ban giám_đốc Công_an TP Đà_Nẵng đã yêu_cầu mở_rộng trang Facebook để tiếp_nhận thông_tin về tai_nạn giao_thông , tình_trạng kẹt xe cũng như những bất_cập khác từ phía người_dân . "',
  'Chúng_tôi đã lập một tổ 7 người , do phó phòng CSGT phụ_trách để tiếp_nhận , lọc thông_tin và xử_lý một_cách nhanh nhất " , đại_tá Ngọc nói .',
  'Hiện Facebook của CSGT Đà_Nẵng đã có gần 2.000 thành_viên .',
  'Nguyễn_Đông   Lập Facebook , CSGT hy_vọng sẽ tiếp_nhận thông_tin và tương_tác nhiều hơn với người_dân .',
  'Ảnh

In [23]:
clean_sentences = pd.Series(sentences[0]).str.replace("[^\w\s]", " ")
clean_sentences[0]

'Đại_tá Lê_Ngọc   Trưởng_phòng CSGT TP Đà_Nẵng   đơn_vị đã chính_thức đưa vào hoạt_động trang Facebook có địa_chỉ   Cảnh_sát giao_thông CATP Đà_Nẵng  '

In [24]:
for i in range(len(clean_sentences)):
  clean_sentences[i] = " ".join(clean_sentences[i].split())
clean_sentences = clean_sentences.replace('', np.nan).dropna()

In [26]:
clean_sentences[0]

'Đại_tá Lê_Ngọc Trưởng_phòng CSGT TP Đà_Nẵng đơn_vị đã chính_thức đưa vào hoạt_động trang Facebook có địa_chỉ Cảnh_sát giao_thông CATP Đà_Nẵng'

## Text rank in test set

In [ ]:
import networkx as nx
df_sumary = []
# split the the text in the articles into sentences
sentences = []

for s in test_df.original:
  sentences.append(sent_tokenize(s))  
for i in range(len(test_df.original)):
  try:
    sentences_index = sentences[i]

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences_index).str.replace("[^\w\s]", " ")
    for i in range(len(clean_sentences)):
      clean_sentences[i] = " ".join(clean_sentences[i].split())
    clean_sentences = clean_sentences.replace('', np.nan).dropna()


    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((300,))
      sentence_vectors.append(v)

    len(sentence_vectors)

    # similarity matrix
    sim_mat = np.zeros([len(sentences_index), len(sentences_index)])
    for i in range(len(sentence_vectors)):
      for j in range(len(sentence_vectors)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]



    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_index)), reverse=True)

    # Specify number of sentences to form the summary
    sn = 3

    # Generate summary
    list_result = []
    str1 = []
    for i in range(sn):
      list_result.append(ranked_sentences[i][1])
      str1 = ''.join(str(e) for e in list_result)
      print(ranked_sentences[i][1])  
    df_sumary.append(list_result)
  except:
    list_result = ['']
    df_sumary.append(list_result)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Năm nay , tôi cũng bán cá vì biết trong Tết mọi người ăn thịt cũng khá nhiều rồi , vì_thế , tôi bán đổi món .
Trong đó , rau bắp_cải , súp_lơ xanh có giá 15.000 đồng / cây , các loại thịt vẫn như ngày thường .
Chủ_đề đó phải nằm trong chương_trình_nghị_sự là do hành_vi của chính_quyền bất_hảo này và mối đe_doạ mà ông Kim_Jong-un gây ra " , ông McMaster nói trong cuộc phỏng_vấn với truyền_thông từ 5 nước châu_Á ông Trump sẽ đến thăm .
Đây sẽ là chuyến công_du đầu_tiên của ông tới khu_vực này với tư_cách tổng_thống Mỹ .
Sẽ là vô_trách_nhiệm nếu không nói về khả_năng thực_hiện nỗ_lực quân_sự trong liên_minh " , Yonhap dẫn lời H.R .
Việt_Nam khẳng_định có chủ_quyền với quần_đảo Hoàng_Sa và Trường_Sa trên Biển_Đông , mọi hoạt_động của các bên mà không được sự cho_phép của Việt_Nam tại hai quần_đảo này đều là phi_pháp và vô_giá_trị .
Bộ_trưởng Quốc_phòng Philippines_Delfin_Lorenzana , người có_mặt trên máy_bay , cho_biết phi cơ không b

In [ ]:
len(df_sumary)

22644

In [ ]:
df = pd.DataFrame(df_sumary, columns = ['predict1', 'predict2','predict3'])

In [ ]:
list_df = []
for i in range(len(df.predict1)):
  temp = str(df.predict1[i]) + str(df.predict2[i])+ str(df.predict3[i])
  list_df.append(temp)

In [ ]:
temp = df.predict1[1] + df.predict2[1]+ df.predict3[1]
type(temp)
list_df.append(temp)

In [ ]:
list_df = pd.DataFrame(list_df, columns = ['predict'])

In [ ]:
list_df.head(5)

,predict
0,Thoả_thuận mới được ký_kết giữa Mỹ và Ấn_Độ ch...
1,Ban_Chấp_hành Trung_ương Đảng Nhân_dân Campuch...
2,Widodo nói ông muốn tuần_tra chung với Austral...
3,"Cảnh_sát phòng cháy chữa_cháy phá tường , dẫn ..."
4,Đoạn video do hành_khách quay lại cho thấy sự ...


In [ ]:
df.head(5)

,predict1,predict2
0,"Điều đáng nói , sức nóng phiên giao_dịch đầu_t...","Có_thể nói , EverGreen như bổ_sung thêm một đị..."
1,Ông Nguyễn_Một : Mặc_dù nguyên_nhân sự_cố hỏng...,"Do_đó , việc đặt ra vấn_đề “ làm gì để những s..."
2,Quan_trọng là tất_cả các quốc_gia phải tôn_trọ...,"Ngoài_ra , Ngoại_trưởng Australia cũng "" hoan_..."
3,"Duterte hé lộ rằng Bắc_Kinh cảnh_báo ông "" có_...","Trước kế_hoạch của ông Duterte , người_phát_ng..."
4,Hai thanh_niên bị_thương điều_trị tại Viện Bỏn...,"Tối 29/7 , Văn_phòng Chính_phủ có văn_bản truy..."


In [ ]:
bigdata = pd.concat([list_df.reset_index(drop=True), test_df.original.reset_index(drop=True)], axis=1)

In [ ]:
bigdata = pd.concat([test_df['file'].reset_index(drop=True), bigdata.reset_index(drop=True)], axis=1)

In [ ]:
bigdata.to_csv('test_predict.csv', encoding='utf-8')

# tính score

In [27]:
!pip install -q rouge_score

In [ ]:
bigdata.head()

,file,predict,original
0,/content/vietnews-master/data/test_tokenized/0...,Thoả_thuận mới được ký_kết giữa Mỹ và Ấn_Độ ch...,Thoả_thuận mới được ký_kết giữa Mỹ và Ấn_Độ ch...
1,/content/vietnews-master/data/test_tokenized/0...,Ban_Chấp_hành Trung_ương Đảng Nhân_dân Campuch...,"Nguyên Tổng_bí_thư Đỗ_Mười . "" Thay_mặt Đảng ,..."
2,/content/vietnews-master/data/test_tokenized/0...,Widodo nói ông muốn tuần_tra chung với Austral...,Tổng_thống Indonesia_Widodo đứng trên boong ch...
3,/content/vietnews-master/data/test_tokenized/0...,"Cảnh_sát phòng cháy chữa_cháy phá tường , dẫn ...",Cảnh_sát phun vòi rồng dập lửa . Video : Bá_Đô...
4,/content/vietnews-master/data/test_tokenized/0...,Đoạn video do hành_khách quay lại cho thấy sự ...,Đồ_hoạ về vụ tai_nạn máy_bay Emirates . Đồ_hoạ...


In [ ]:
from rouge_score import rouge_scorer
score_list = []
for i in range(len(bigdata.original)):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(str(bigdata.predict[i]),str(bigdata.original[i]))
  score_list.append(scores)

In [ ]:
df_score_test = pd.DataFrame(score_list, columns = ['score'])

In [ ]:
score_list[6]

{'rouge1': Score(precision=0.28837209302325584, recall=1.0, fmeasure=0.4476534296028881),
 'rouge2': Score(precision=0.2867132867132867, recall=1.0, fmeasure=0.4456521739130435),
 'rougeL': Score(precision=0.2441860465116279, recall=0.8467741935483871, fmeasure=0.3790613718411553)}

In [ ]:
score_r1 = []
score_rl = []
score_r2 = []
for i in range(len(score_list)):
  score_r1.append(score_list[i]['rouge1'].fmeasure)
  score_r2.append(score_list[i]['rouge2'].fmeasure)
  score_rl.append(score_list[i]['rougeL'].fmeasure)

In [ ]:
import statistics
print(statistics.mean(score_r1))
print(statistics.mean(score_r2))
print(statistics.mean(score_rl))

0.45200444554247593
0.447521506032258
0.3774385023995794


- **2 câu**
- R1: 0.34111518169308247
- R2: 0.33769186213340274
- RL: 0.3080631855000541
- **3 câu**


- R1: 0.45200444554247593
- R2: 0.447521506032258
- RL: 0.3774385023995794 

In [30]:
from rouge_score import rouge_scorer
score_list = []
for i in range(1):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score("Võ_Minh_Trí","võ_minh_trí")
  score_list.append(scores)

In [31]:
score_list

[{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0),
  'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0),
  'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}]

# train set

In [ ]:
import networkx as nx
df_train_sumary = []
# split the the text in the articles into sentences
sentences = []

for s in train_df.original:
  sentences.append(sent_tokenize(s))  
for i in range(len(train_df.original)):
  try:
    sentences_index = sentences[i]

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences_index).str.replace("[^\w\s]", " ")
    for i in range(len(clean_sentences)):
      clean_sentences[i] = " ".join(clean_sentences[i].split())
    clean_sentences = clean_sentences.replace('', np.nan).dropna()


    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((300,))
      sentence_vectors.append(v)

    len(sentence_vectors)

    # similarity matrix
    sim_mat = np.zeros([len(sentences_index), len(sentences_index)])
    for i in range(len(sentence_vectors)):
      for j in range(len(sentence_vectors)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]



    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_index)), reverse=True)

    # Specify number of sentences to form the summary
    sn = 1

    # Generate summary
    list_result = []
    str1 = []
    for i in range(sn):
      list_result.append(ranked_sentences[i][1])
      str1 = ''.join(str(e) for e in list_result)
      print(ranked_sentences[i][1])  
    df_sumary.append(list_result)
  except:
    list_result = ['']
    df_sumary.append(list_result)

In [ ]:
df_train_pre = pd.DataFrame(df_train_sumary, columns = ['predict'])

In [ ]:
bigdata_train = pd.concat([df_train_pre.reset_index(drop=True), train_df.original.reset_index(drop=True)], axis=1)

In [ ]:
bigdata_train = pd.concat([train_df['file'].reset_index(drop=True), bigdata.reset_index(drop=True)], axis=1)

In [ ]:
bigdata.to_csv('test_predict_1c.csv', encoding='utf8')

In [ ]:
from rouge_score import rouge_scorer
score_list_train = []
for i in range(len(bigdata_train.original)):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  scores = scorer.score(bigdata_train.predict[i],bigdata_train.original[i])
  score_list_train.append(scores)

In [ ]:
score_list_train

In [ ]:
score_r1 = []
score_rl = []
for i in range(5000):
  score_r1.append(score_list_train[i]['rouge1'].fmeasure)
  score_rl.append(score_list_train[i]['rougeL'].fmeasure)

In [ ]:
import statistics
print(statistics.mean(score_r1))
print(statistics.mean(score_rl))

0.19911064978230728
0.19911064978230728


In [ ]:
a = [1,2,3,4]
statistics.mean(a)

2.5